# Chapter 5 - Exercise 3: Cell Growth Kinetics & Culture Modes

## Learning Objectives

After completing this exercise, you will be able to:

1. **Model** exponential and logistic cell growth using differential equations
2. **Compare** batch, fed-batch, and continuous culture modes
3. **Calculate** doubling time, specific growth rate, and productivity
4. **Understand** nutrient limitation and growth inhibition
5. **Optimize** culture strategies for different bioprocessing goals

---

## Background: Cell Growth Models

### Growth Phases in Batch Culture

When cells are inoculated into fresh medium, they typically progress through distinct phases:

1. **Lag phase**: Adaptation period, no/minimal growth
2. **Exponential (log) phase**: Rapid growth at maximum rate μ_max
3. **Stationary phase**: Growth ceases due to nutrient depletion or inhibitor accumulation
4. **Death phase**: Cell viability declines

### Exponential Growth

During exponential phase:

$$\frac{dX}{dt} = \mu X$$

Where:
- **X** = cell concentration (cells/mL)
- **μ** = specific growth rate (h⁻¹)
- **t** = time (hours)

Solution: $X(t) = X_0 e^{\mu t}$

**Doubling time:** $t_d = \frac{\ln(2)}{\mu} = \frac{0.693}{\mu}$

### Logistic Growth (Monod Model)

Growth slows as nutrients deplete:

$$\mu = \mu_{max} \frac{S}{K_S + S}$$

$$\frac{dX}{dt} = \mu X - k_d X$$

$$\frac{dS}{dt} = -\frac{1}{Y_{X/S}} \mu X$$

Where:
- **S** = substrate (nutrient) concentration (g/L)
- **K_S** = half-saturation constant (g/L)
- **k_d** = death rate constant (h⁻¹)
- **Y_X/S** = yield coefficient (cells/g substrate)

---

## Culture Modes

### 1. Batch Culture
**Operation:** Cells + medium → grow → harvest  
**Pros:** Simple, contained, low contamination risk  
**Cons:** Limited productivity, nutrient depletion, waste accumulation  
**Best for:** Small-scale, proof-of-concept, GMP production

### 2. Fed-Batch Culture
**Operation:** Start batch + periodic nutrient feeding (no removal)  
**Pros:** Higher cell density, extended production, controlled growth  
**Cons:** Volume increases, waste accumulates  
**Best for:** High-density cultures, antibody/protein production

### 3. Continuous Culture (Chemostat)
**Operation:** Continuous feed + continuous removal at dilution rate D  
**Pros:** Steady-state operation, highest productivity, unlimited duration  
**Cons:** Contamination risk, complex operation, genetic drift  
**Best for:** Long-term studies, metabolite production, research

At steady state: $\mu = D$

### 4. Perfusion Culture
**Operation:** Continuous medium exchange with cell retention  
**Pros:** Very high density, long duration, waste removal  
**Cons:** Most complex, expensive  
**Best for:** Cell therapy, continuous viral production

---

## Setup: Import Required Libraries

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact, widgets, Layout
from IPython.display import display, HTML
from scipy.integrate import odeint
import seaborn as sns

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully!")
print("Ready to model cell growth...")

## Cell Type Parameters Database

In [ ]:
# Cell type growth parameters
CELL_GROWTH_PARAMS = {
    'CHO cells': {
        'mu_max': 0.035,  # h⁻¹ (doubling ~20h)
        'X_max': 1e7,  # cells/mL maximum density
        'K_S': 0.1,  # g/L glucose
        'Y_XS': 3e8,  # cells/g glucose
        'k_d': 0.002,  # h⁻¹ death rate
        'qS': 2e-10,  # g glucose/cell/h
        'lag_time': 6,  # hours
    },
    'HEK293': {
        'mu_max': 0.042,  # h⁻¹ (doubling ~16.5h)
        'X_max': 6e6,
        'K_S': 0.15,
        'Y_XS': 2.5e8,
        'k_d': 0.003,
        'qS': 2.5e-10,
        'lag_time': 4,
    },
    'Hybridoma': {
        'mu_max': 0.028,  # h⁻¹ (doubling ~25h)
        'X_max': 5e6,
        'K_S': 0.2,
        'Y_XS': 2e8,
        'k_d': 0.004,
        'qS': 3e-10,
        'lag_time': 8,
    },
    'E. coli': {
        'mu_max': 0.7,  # h⁻¹ (doubling ~1h) - very fast!
        'X_max': 5e9,  # Much higher density
        'K_S': 0.01,  # Very efficient
        'Y_XS': 5e9,
        'k_d': 0.001,
        'qS': 1e-12,
        'lag_time': 1,
    },
    'MSCs': {
        'mu_max': 0.025,  # h⁻¹ (doubling ~28h)
        'X_max': 8e5,  # Lower density (adherent)
        'K_S': 0.3,
        'Y_XS': 1.5e8,
        'k_d': 0.001,  # Very low death rate
        'qS': 4e-10,
        'lag_time': 12,  # Long adaptation
    },
    'T-cells': {
        'mu_max': 0.031,  # h⁻¹ (doubling ~22h)
        'X_max': 4e6,
        'K_S': 0.25,
        'Y_XS': 2.2e8,
        'k_d': 0.005,  # Higher death rate
        'qS': 3.5e-10,
        'lag_time': 10,
    },
}

print("✓ Cell growth parameters loaded")
print(f"  - {len(CELL_GROWTH_PARAMS)} cell types")

# Display doubling times
print("\n📊 Doubling Times:")
for cell_type, params in CELL_GROWTH_PARAMS.items():
    t_d = 0.693 / params['mu_max']
    print(f"  {cell_type:15s}: {t_d:.1f} hours")

## Growth Model Functions

In [ ]:
def batch_culture(t, X0, S0, params):
    """
    Simulate batch culture with Monod kinetics.
    
    Returns: X (cells/mL), S (g/L substrate), mu (h⁻¹)
    """
    
    def model(y, t):
        X, S = y
        
        # Monod growth rate
        mu = params['mu_max'] * S / (params['K_S'] + S)
        
        # Apply lag phase (simple exponential adaptation)
        lag_factor = 1 - np.exp(-t/params['lag_time'])
        mu_eff = mu * lag_factor
        
        # Apply maximum density limitation
        mu_eff = mu_eff * (1 - X/params['X_max'])
        
        # Differential equations
        dXdt = mu_eff * X - params['k_d'] * X
        dSdt = -params['qS'] * X
        
        return [dXdt, dSdt]
    
    # Solve
    y0 = [X0, S0]
    sol = odeint(model, y0, t)
    X = sol[:, 0]
    S = sol[:, 1]
    
    # Calculate instantaneous growth rate
    mu = np.array([params['mu_max'] * s / (params['K_S'] + s) for s in S])
    
    return X, S, mu


def fed_batch_culture(t, X0, S0, feed_rate, feed_start, params):
    """
    Simulate fed-batch culture with glucose feeding.
    
    feed_rate: g/L/h glucose addition rate
    feed_start: time (h) when feeding begins
    """
    
    def model(y, t):
        X, S = y
        
        # Monod growth rate
        mu = params['mu_max'] * S / (params['K_S'] + S)
        
        # Apply lag phase
        lag_factor = 1 - np.exp(-t/params['lag_time'])
        mu_eff = mu * lag_factor
        
        # Apply maximum density limitation
        mu_eff = mu_eff * (1 - X/params['X_max'])
        
        # Feeding rate (starts at feed_start time)
        F = feed_rate if t >= feed_start else 0
        
        # Differential equations
        dXdt = mu_eff * X - params['k_d'] * X
        dSdt = -params['qS'] * X + F  # Feeding adds glucose
        
        return [dXdt, dSdt]
    
    y0 = [X0, S0]
    sol = odeint(model, y0, t)
    X = sol[:, 0]
    S = sol[:, 1]
    
    mu = np.array([params['mu_max'] * s / (params['K_S'] + s) for s in S])
    
    return X, S, mu


def continuous_culture(D, S_feed, params):
    """
    Calculate steady-state for continuous culture (chemostat).
    
    D: dilution rate (h⁻¹) = F/V
    S_feed: feed substrate concentration (g/L)
    
    At steady state: μ = D
    """
    
    # Steady-state substrate
    S_ss = (params['K_S'] * D) / (params['mu_max'] - D)
    
    if S_ss < 0 or D >= params['mu_max']:
        # Washout condition
        return 0, S_feed, 0
    
    # Steady-state cell density
    X_ss = params['Y_XS'] * (S_feed - S_ss)
    
    # Productivity
    productivity = D * X_ss
    
    return X_ss, S_ss, productivity


def calculate_metrics(t, X, S):
    """
    Calculate performance metrics from growth data.
    """
    
    # Final values
    X_final = X[-1]
    S_final = S[-1]
    duration = t[-1]
    
    # Average productivity
    avg_productivity = X_final / duration  # cells/mL/h
    
    # Maximum productivity (instantaneous)
    dXdt = np.gradient(X, t)
    max_productivity = np.max(dXdt)
    
    # Substrate consumed
    S_consumed = S[0] - S_final
    
    # Observed yield
    Y_obs = (X_final - X[0]) / S_consumed if S_consumed > 0 else 0
    
    return {
        'X_final': X_final,
        'S_final': S_final,
        'Duration': duration,
        'Avg_Productivity': avg_productivity,
        'Max_Productivity': max_productivity,
        'S_consumed': S_consumed,
        'Yield': Y_obs,
    }

print("✓ Growth model functions defined")

---

## Interactive Tool 1: Batch Culture Simulator

### Instructions:

1. **Select cell type** to use its characteristic growth parameters
2. **Set initial conditions:**
   - Inoculum density (X₀)
   - Initial glucose concentration (S₀)
3. **Choose culture duration**
4. **Observe:**
   - Cell growth curve with distinct phases
   - Nutrient depletion
   - Growth rate dynamics
   - Performance metrics

In [ ]:
def interactive_batch_culture(cell_type='CHO cells',
                             X0=2e5,
                             S0=4.5,
                             duration_hours=120):
    """
    Interactive batch culture simulator.
    """
    
    # Get parameters
    params = CELL_GROWTH_PARAMS[cell_type]
    
    # Time array
    t = np.linspace(0, duration_hours, 500)
    
    # Simulate
    X, S, mu = batch_culture(t, X0, S0, params)
    
    # Calculate metrics
    metrics = calculate_metrics(t, X, S)
    
    # Create visualizations
    fig = plt.figure(figsize=(16, 10))
    
    # 1. Cell growth curve (with phases)
    ax1 = plt.subplot(2, 3, 1)
    ax1.semilogy(t, X, 'b-', linewidth=2.5, label='Cell Density')
    ax1.axhline(y=params['X_max'], color='r', linestyle='--', 
               label=f"Max Density ({params['X_max']:.1e})")
    
    # Mark phases
    lag_end = params['lag_time']
    ax1.axvspan(0, lag_end, alpha=0.2, color='yellow', label='Lag Phase')
    
    # Find exponential phase (where mu > 80% of mu_max)
    exp_idx = np.where(mu > 0.8 * params['mu_max'])[0]
    if len(exp_idx) > 0:
        exp_start = t[exp_idx[0]]
        exp_end = t[exp_idx[-1]]
        ax1.axvspan(exp_start, exp_end, alpha=0.2, color='green', label='Exponential Phase')
    
    ax1.set_xlabel('Time (hours)', fontsize=11, fontweight='bold')
    ax1.set_ylabel('Cell Density (cells/mL)', fontsize=11, fontweight='bold')
    ax1.set_title(f'Batch Culture Growth - {cell_type}', fontsize=13, fontweight='bold')
    ax1.legend(loc='best')
    ax1.grid(True, alpha=0.3, which='both')
    
    # 2. Glucose depletion
    ax2 = plt.subplot(2, 3, 2)
    ax2.plot(t, S, 'orange', linewidth=2.5, label='Glucose')
    ax2.axhline(y=params['K_S'], color='red', linestyle='--', 
               label=f'K_S = {params["K_S"]} g/L')
    ax2.fill_between(t, 0, params['K_S'], alpha=0.2, color='red', 
                     label='Growth-limiting region')
    ax2.set_xlabel('Time (hours)', fontsize=11, fontweight='bold')
    ax2.set_ylabel('Glucose Concentration (g/L)', fontsize=11, fontweight='bold')
    ax2.set_title('Nutrient Depletion', fontsize=13, fontweight='bold')
    ax2.legend(loc='best')
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim([0, S0 * 1.1])
    
    # 3. Specific growth rate
    ax3 = plt.subplot(2, 3, 3)
    ax3.plot(t, mu, 'purple', linewidth=2.5)
    ax3.axhline(y=params['mu_max'], color='green', linestyle='--', 
               label=f"μ_max = {params['mu_max']:.3f} h⁻¹")
    ax3.set_xlabel('Time (hours)', fontsize=11, fontweight='bold')
    ax3.set_ylabel('Specific Growth Rate μ (h⁻¹)', fontsize=11, fontweight='bold')
    ax3.set_title('Growth Rate Dynamics', fontsize=13, fontweight='bold')
    ax3.legend(loc='best')
    ax3.grid(True, alpha=0.3)
    
    # 4. Cell-Substrate relationship
    ax4 = plt.subplot(2, 3, 4)
    ax4.plot(S, X, 'b-', linewidth=2)
    ax4.plot(S0, X0, 'go', markersize=10, label='Start')
    ax4.plot(S[-1], X[-1], 'ro', markersize=10, label='End')
    ax4.set_xlabel('Glucose (g/L)', fontsize=11, fontweight='bold')
    ax4.set_ylabel('Cell Density (cells/mL)', fontsize=11, fontweight='bold')
    ax4.set_title('Phase Portrait: Cells vs Substrate', fontsize=13, fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    ax4.set_yscale('log')
    
    # 5. Productivity over time
    ax5 = plt.subplot(2, 3, 5)
    productivity = np.gradient(X, t)  # Instantaneous productivity
    ax5.plot(t, productivity, 'green', linewidth=2)
    ax5.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax5.set_xlabel('Time (hours)', fontsize=11, fontweight='bold')
    ax5.set_ylabel('Productivity (cells/mL/h)', fontsize=11, fontweight='bold')
    ax5.set_title('Instantaneous Productivity', fontsize=13, fontweight='bold')
    ax5.grid(True, alpha=0.3)
    
    # Mark maximum
    max_idx = np.argmax(productivity)
    ax5.plot(t[max_idx], productivity[max_idx], 'r*', markersize=15, 
            label=f'Max: {productivity[max_idx]:.2e}')
    ax5.legend()
    
    # 6. Performance metrics table
    ax6 = plt.subplot(2, 3, 6)
    ax6.axis('off')
    
    # Calculate doubling time
    t_d = 0.693 / params['mu_max']
    
    metrics_text = f"""
    BATCH CULTURE SUMMARY
    {'='*45}
    
    CELL TYPE: {cell_type}
    
    KINETIC PARAMETERS
    μ_max:  {params['mu_max']:.3f} h⁻¹
    Doubling time: {t_d:.1f} hours
    K_S:    {params['K_S']:.2f} g/L
    X_max:  {params['X_max']:.2e} cells/mL
    
    INITIAL CONDITIONS
    X₀:     {X0:.2e} cells/mL
    S₀:     {S0:.2f} g/L glucose
    
    FINAL RESULTS
    X_final: {metrics['X_final']:.2e} cells/mL
    S_final: {metrics['S_final']:.2f} g/L
    Fold increase: {metrics['X_final']/X0:.1f}x
    
    PERFORMANCE
    Duration: {duration_hours:.0f} hours
    Avg productivity: {metrics['Avg_Productivity']:.2e} cells/mL/h
    Max productivity: {metrics['Max_Productivity']:.2e} cells/mL/h
    
    EFFICIENCY
    Glucose consumed: {metrics['S_consumed']:.2f} g/L
    Yield: {metrics['Yield']:.2e} cells/g glucose
    """
    
    ax6.text(0.1, 0.95, metrics_text, transform=ax6.transAxes,
            fontsize=9, verticalalignment='top', family='monospace',
            bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
    
    plt.tight_layout()
    plt.show()
    
    # Print analysis
    print("\n" + "="*80)
    print("📊 GROWTH PHASE ANALYSIS")
    print("="*80)
    
    print(f"\n1. LAG PHASE (0 - {lag_end:.1f} hours)")
    print(f"   Cells adapting to new environment")
    print(f"   Minimal growth observed")
    
    if len(exp_idx) > 0:
        print(f"\n2. EXPONENTIAL PHASE ({exp_start:.1f} - {exp_end:.1f} hours)")
        print(f"   Duration: {exp_end - exp_start:.1f} hours")
        print(f"   Growth rate: ~{params['mu_max']:.3f} h⁻¹")
        print(f"   Doubling time: ~{t_d:.1f} hours")
        exp_X_start = X[exp_idx[0]]
        exp_X_end = X[exp_idx[-1]]
        doublings = np.log2(exp_X_end / exp_X_start)
        print(f"   Population doublings: {doublings:.1f}")
    
    # Find when growth rate drops below 10% of max
    stat_idx = np.where(mu < 0.1 * params['mu_max'])[0]
    if len(stat_idx) > 0:
        stat_start = t[stat_idx[0]]
        print(f"\n3. STATIONARY PHASE (starts ~{stat_start:.1f} hours)")
        print(f"   Glucose: {S[stat_idx[0]]:.2f} g/L (depleted)")
        print(f"   Cell density plateaus at {X[stat_idx[0]]:.2e} cells/mL")
    
    print(f"\n4. OVERALL PERFORMANCE")
    total_cells = metrics['X_final'] * 1  # Assuming 1L
    print(f"   Total cells produced: {total_cells:.2e}")
    print(f"   Process time: {duration_hours} hours")
    print(f"   Space-time yield: {metrics['Avg_Productivity']:.2e} cells/mL/h")
    
    print("\n" + "="*80)

print("✓ Batch culture simulator ready")

---

## 🎮 TOOL 1: Batch Culture - Run This Cell!

In [ ]:
interact(interactive_batch_culture,
         cell_type=widgets.Dropdown(
             options=list(CELL_GROWTH_PARAMS.keys()),
             value='CHO cells',
             description='Cell Type:',
             style={'description_width': '150px'},
             layout=Layout(width='400px')
         ),
         X0=widgets.FloatLogSlider(
             value=2e5,
             min=4,  # 10^4
             max=6,  # 10^6
             step=0.1,
             description='X₀ (cells/mL):',
             readout_format='.2e',
             style={'description_width': '150px'},
             layout=Layout(width='500px')
         ),
         S0=widgets.FloatSlider(
             value=4.5,
             min=1.0,
             max=10.0,
             step=0.5,
             description='S₀ Glucose (g/L):',
             style={'description_width': '150px'},
             layout=Layout(width='500px')
         ),
         duration_hours=widgets.IntSlider(
             value=120,
             min=24,
             max=240,
             step=12,
             description='Duration (hours):',
             style={'description_width': '150px'},
             layout=Layout(width='500px')
         ));

---

## Interactive Tool 2: Culture Mode Comparison

Compare batch, fed-batch, and continuous culture side-by-side.

In [ ]:
def compare_culture_modes(cell_type='CHO cells',
                         X0=2e5,
                         S0=4.5,
                         duration_hours=168,
                         fed_batch_rate=0.05,
                         feed_start_time=48,
                         continuous_dilution_rate=0.02):
    """
    Compare batch, fed-batch, and continuous culture.
    """
    
    params = CELL_GROWTH_PARAMS[cell_type]
    t = np.linspace(0, duration_hours, 500)
    
    # Simulate batch
    X_batch, S_batch, mu_batch = batch_culture(t, X0, S0, params)
    
    # Simulate fed-batch
    X_fed, S_fed, mu_fed = fed_batch_culture(t, X0, S0, fed_batch_rate, 
                                             feed_start_time, params)
    
    # Continuous culture (steady-state)
    X_cont, S_cont, prod_cont = continuous_culture(continuous_dilution_rate, S0, params)
    
    # Create comparison plot
    fig = plt.figure(figsize=(16, 10))
    
    # 1. Cell density comparison
    ax1 = plt.subplot(2, 3, 1)
    ax1.semilogy(t, X_batch, 'b-', linewidth=2.5, label='Batch')
    ax1.semilogy(t, X_fed, 'g-', linewidth=2.5, label='Fed-Batch')
    if X_cont > 0:
        ax1.axhline(y=X_cont, color='orange', linestyle='--', linewidth=2.5, 
                   label=f'Continuous (D={continuous_dilution_rate:.3f})')
    ax1.set_xlabel('Time (hours)', fontsize=11, fontweight='bold')
    ax1.set_ylabel('Cell Density (cells/mL)', fontsize=11, fontweight='bold')
    ax1.set_title('Cell Growth Comparison', fontsize=13, fontweight='bold')
    ax1.legend(loc='best')
    ax1.grid(True, alpha=0.3, which='both')
    
    # 2. Glucose comparison
    ax2 = plt.subplot(2, 3, 2)
    ax2.plot(t, S_batch, 'b-', linewidth=2.5, label='Batch')
    ax2.plot(t, S_fed, 'g-', linewidth=2.5, label='Fed-Batch')
    if S_cont > 0:
        ax2.axhline(y=S_cont, color='orange', linestyle='--', linewidth=2.5, 
                   label='Continuous')
    ax2.axvline(x=feed_start_time, color='gray', linestyle=':', 
               label='Feeding starts', alpha=0.7)
    ax2.set_xlabel('Time (hours)', fontsize=11, fontweight='bold')
    ax2.set_ylabel('Glucose (g/L)', fontsize=11, fontweight='bold')
    ax2.set_title('Nutrient Dynamics', fontsize=13, fontweight='bold')
    ax2.legend(loc='best')
    ax2.grid(True, alpha=0.3)
    
    # 3. Cumulative cell production
    ax3 = plt.subplot(2, 3, 3)
    # For batch and fed-batch: total cells = X * V (assuming V=1L)
    cumulative_batch = X_batch
    cumulative_fed = X_fed
    # For continuous: cells harvested over time = D * X_ss * t
    if X_cont > 0:
        cumulative_cont = continuous_dilution_rate * X_cont * t
    else:
        cumulative_cont = np.zeros_like(t)
    
    ax3.plot(t, cumulative_batch, 'b-', linewidth=2.5, label='Batch')
    ax3.plot(t, cumulative_fed, 'g-', linewidth=2.5, label='Fed-Batch')
    ax3.plot(t, cumulative_cont, 'orange', linestyle='--', linewidth=2.5, 
            label='Continuous')
    ax3.set_xlabel('Time (hours)', fontsize=11, fontweight='bold')
    ax3.set_ylabel('Cumulative Cells (cells/mL)', fontsize=11, fontweight='bold')
    ax3.set_title('Total Cell Production', fontsize=13, fontweight='bold')
    ax3.legend(loc='best')
    ax3.grid(True, alpha=0.3)
    ax3.set_yscale('log')
    
    # 4. Productivity comparison
    ax4 = plt.subplot(2, 3, 4)
    prod_batch = np.gradient(X_batch, t)
    prod_fed = np.gradient(X_fed, t)
    
    ax4.plot(t, prod_batch, 'b-', linewidth=2, label='Batch')
    ax4.plot(t, prod_fed, 'g-', linewidth=2, label='Fed-Batch')
    if X_cont > 0:
        ax4.axhline(y=prod_cont, color='orange', linestyle='--', linewidth=2, 
                   label='Continuous')
    ax4.set_xlabel('Time (hours)', fontsize=11, fontweight='bold')
    ax4.set_ylabel('Productivity (cells/mL/h)', fontsize=11, fontweight='bold')
    ax4.set_title('Instantaneous Productivity', fontsize=13, fontweight='bold')
    ax4.legend(loc='best')
    ax4.grid(True, alpha=0.3)
    
    # 5. Performance metrics bar chart
    ax5 = plt.subplot(2, 3, 5)
    
    metrics_batch = calculate_metrics(t, X_batch, S_batch)
    metrics_fed = calculate_metrics(t, X_fed, S_fed)
    
    modes = ['Batch', 'Fed-Batch', 'Continuous']
    final_X = [X_batch[-1], X_fed[-1], X_cont if X_cont > 0 else 0]
    avg_prod = [metrics_batch['Avg_Productivity'], 
               metrics_fed['Avg_Productivity'],
               prod_cont if X_cont > 0 else 0]
    
    x_pos = np.arange(len(modes))
    width = 0.35
    
    bars1 = ax5.bar(x_pos - width/2, final_X, width, label='Final Density', alpha=0.8)
    ax5_twin = ax5.twinx()
    bars2 = ax5_twin.bar(x_pos + width/2, avg_prod, width, label='Avg Productivity', 
                        color='orange', alpha=0.8)
    
    ax5.set_xlabel('Culture Mode', fontsize=11, fontweight='bold')
    ax5.set_ylabel('Final Density (cells/mL)', fontsize=10, fontweight='bold')
    ax5_twin.set_ylabel('Avg Productivity (cells/mL/h)', fontsize=10, fontweight='bold')
    ax5.set_title('Performance Comparison', fontsize=13, fontweight='bold')
    ax5.set_xticks(x_pos)
    ax5.set_xticklabels(modes)
    ax5.legend(loc='upper left')
    ax5_twin.legend(loc='upper right')
    ax5.set_yscale('log')
    ax5.grid(True, alpha=0.3, axis='y')
    
    # 6. Summary table
    ax6 = plt.subplot(2, 3, 6)
    ax6.axis('off')
    
    summary_text = f"""
    CULTURE MODE COMPARISON
    {'='*50}
    
    BATCH CULTURE
    Final density: {X_batch[-1]:.2e} cells/mL
    Avg productivity: {metrics_batch['Avg_Productivity']:.2e}
    Glucose used: {metrics_batch['S_consumed']:.2f} g/L
    
    FED-BATCH CULTURE
    Final density: {X_fed[-1]:.2e} cells/mL
    Avg productivity: {metrics_fed['Avg_Productivity']:.2e}
    Feed rate: {fed_batch_rate:.3f} g/L/h
    Feed start: {feed_start_time} hours
    Improvement: {X_fed[-1]/X_batch[-1]:.1f}x vs batch
    
    CONTINUOUS CULTURE
    Steady-state X: {X_cont:.2e} cells/mL
    Steady-state S: {S_cont:.2f} g/L
    Dilution rate: {continuous_dilution_rate:.3f} h⁻¹
    Productivity: {prod_cont:.2e} cells/mL/h
    {'Washout!' if X_cont == 0 else 'Stable operation'}
    
    BEST FOR:
    Highest density → Fed-Batch
    Continuous output → Continuous
    Simplicity/GMP → Batch
    """
    
    ax6.text(0.05, 0.95, summary_text, transform=ax6.transAxes,
            fontsize=9, verticalalignment='top', family='monospace',
            bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.5))
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed comparison
    print("\n" + "="*80)
    print("📊 DETAILED MODE COMPARISON")
    print("="*80)
    
    print(f"\n1. BATCH CULTURE")
    print(f"   ✓ Simple operation, minimal equipment")
    print(f"   ✓ Closed system → low contamination risk")
    print(f"   ✗ Limited by initial nutrients")
    print(f"   Final yield: {X_batch[-1]:.2e} cells/mL")
    
    print(f"\n2. FED-BATCH CULTURE")
    print(f"   ✓ {X_fed[-1]/X_batch[-1]:.1f}x higher density than batch")
    print(f"   ✓ Extended production phase")
    print(f"   ✓ Control over growth rate via feeding")
    print(f"   ✗ Volume increases (may need larger vessel)")
    print(f"   Final yield: {X_fed[-1]:.2e} cells/mL")
    
    print(f"\n3. CONTINUOUS CULTURE")
    if X_cont > 0:
        print(f"   ✓ Steady-state operation indefinitely")
        print(f"   ✓ Constant productivity: {prod_cont:.2e} cells/mL/h")
        print(f"   ✓ Cells at exponential growth continuously")
        print(f"   ✗ More complex operation")
        print(f"   ✗ Contamination risk over long durations")
        
        # Calculate when continuous overtakes batch
        overtake_time = X_batch[-1] / prod_cont
        print(f"   Break-even vs batch: ~{overtake_time:.0f} hours")
    else:
        print(f"   ⚠️  WASHOUT CONDITION!")
        print(f"   Dilution rate ({continuous_dilution_rate:.3f}) > μ_max ({params['mu_max']:.3f})")
        print(f"   Cells are removed faster than they can grow")
        print(f"   Reduce dilution rate below {params['mu_max']:.3f} h⁻¹")
    
    print(f"\n4. RECOMMENDATION")
    if X_fed[-1] > 2 * X_batch[-1]:
        print(f"   → Fed-batch offers significant advantage ({X_fed[-1]/X_batch[-1]:.1f}x improvement)")
    else:
        print(f"   → Batch may be sufficient (simpler operation)")
    
    print("\n" + "="*80)

print("✓ Culture mode comparison ready")

---

## 🎮 TOOL 2: Culture Mode Comparison - Run This Cell!

In [ ]:
interact(compare_culture_modes,
         cell_type=widgets.Dropdown(
             options=list(CELL_GROWTH_PARAMS.keys()),
             value='CHO cells',
             description='Cell Type:',
             style={'description_width': '150px'},
             layout=Layout(width='400px')
         ),
         X0=widgets.FloatLogSlider(
             value=2e5,
             min=4,
             max=6,
             step=0.1,
             description='X₀ (cells/mL):',
             readout_format='.2e',
             style={'description_width': '150px'},
             layout=Layout(width='500px')
         ),
         S0=widgets.FloatSlider(
             value=4.5,
             min=1.0,
             max=10.0,
             step=0.5,
             description='S₀ Glucose (g/L):',
             style={'description_width': '150px'},
             layout=Layout(width='500px')
         ),
         duration_hours=widgets.IntSlider(
             value=168,
             min=48,
             max=336,
             step=24,
             description='Duration (hours):',
             style={'description_width': '150px'},
             layout=Layout(width='500px')
         ),
         fed_batch_rate=widgets.FloatSlider(
             value=0.05,
             min=0.01,
             max=0.2,
             step=0.01,
             description='Feed Rate (g/L/h):',
             style={'description_width': '150px'},
             layout=Layout(width='500px')
         ),
         feed_start_time=widgets.IntSlider(
             value=48,
             min=12,
             max=120,
             step=12,
             description='Feed Start (h):',
             style={'description_width': '150px'},
             layout=Layout(width='500px')
         ),
         continuous_dilution_rate=widgets.FloatSlider(
             value=0.02,
             min=0.005,
             max=0.05,
             step=0.005,
             description='Dilution Rate D (h⁻¹):',
             style={'description_width': '150px'},
             layout=Layout(width='500px')
         ));

---

## 📝 Student Exercises

### Exercise 1: Understanding Growth Kinetics
**Use CHO cells in batch culture:**

1. Start with X₀ = 2×10⁵, S₀ = 4.5 g/L. What is the final cell density?
2. Double the initial glucose to 9.0 g/L. Does final cell density double?
3. What limits the maximum cell density? (Hint: look at the phase portrait)
4. Calculate how many population doublings occur during exponential phase

### Exercise 2: Cell Type Comparison
**Compare E. coli vs CHO cells (same conditions: X₀ = 2×10⁵, S₀ = 4.5 g/L):**

1. Which reaches higher density faster?
2. Compare their doubling times
3. Why does E. coli grow so much faster?
4. Which would you choose for rapid protein production?

### Exercise 3: Fed-Batch Optimization
**Using CHO cells, optimize fed-batch conditions:**

1. Start with batch (no feeding). Note final density.
2. Add feeding at 0.05 g/L/h starting at 48h. How much improvement?
3. Try different feed start times (24h, 48h, 72h). When is optimal?
4. Increase feed rate to 0.1 g/L/h. Does more feeding always help?

### Exercise 4: Continuous Culture
**Explore chemostat operation with CHO cells:**

1. Set D = 0.020 h⁻¹. What is steady-state cell density?
2. Increase D to 0.030 h⁻¹. What happens?
3. Find the maximum dilution rate before washout (μ_max)
4. At what D is productivity (D × X_ss) maximized?

### Exercise 5: Mode Selection
**For each scenario, recommend best culture mode:**

1. **Clinical trial** - need 1×10⁹ cells, GMP compliant, one batch
2. **Research** - need steady supply of exponential cells for 2 months
3. **Antibody production** - maximize total protein over 7 days
4. **Cell therapy** - need 5×10⁸ cells/week continuously

### Exercise 6: Scale-Up Considerations
**Using Hybridoma cells:**

1. In batch mode, how long to reach 5×10⁶ cells/mL?
2. If you need 5×10¹¹ total cells, what volume batch reactor is needed?
3. Compare: 5× 100L batches vs. 1× 500L batch (consider turnaround time)
4. Would continuous culture be more productive?

---

## 💭 Discussion Questions

1. **Batch vs Continuous:**
   - Why is batch culture still dominant in bioprocessing despite lower productivity?
   - When does continuous culture make economic sense?

2. **Fed-Batch Strategy:**
   - How do you decide when to start feeding?
   - What are risks of overfeeding? Underfeeding?

3. **Genetic Stability:**
   - Why might continuous culture lead to genetic drift?
   - How many generations can you safely culture cells?

4. **Regulatory Perspective:**
   - Why do regulators prefer batch processing for pharmaceuticals?
   - What additional controls are needed for continuous manufacturing?

5. **Future Trends:**
   - How might perfusion systems combine benefits of fed-batch and continuous?
   - Role of automation and AI in optimizing culture modes?

---

## 📚 Key Takeaways

✅ **Exponential growth** follows $X(t) = X_0 e^{\mu t}$ with doubling time $t_d = 0.693/\mu$

✅ **Monod kinetics** describe nutrient-limited growth: $\mu = \mu_{max} \frac{S}{K_S + S}$

✅ **Culture mode trade-offs:**
- **Batch**: Simple, contained, limited productivity
- **Fed-batch**: Higher density, extended production, increasing volume
- **Continuous**: Steady-state, highest productivity, complexity/contamination risk
- **Perfusion**: Highest density, waste removal, most complex

✅ **Fed-batch advantages:**
- Achieve 2-5× higher cell density than batch
- Control growth rate via nutrient feeding
- Standard for antibody/protein production

✅ **Continuous culture:**
- At steady-state: μ = D (dilution rate)
- Washout occurs when D > μ_max
- Optimal productivity at intermediate D

✅ **Cell type differences:**
- E. coli: Fast (t_d ~1h), high density (10⁹/mL)
- Mammalian cells: Slower (t_d ~20-30h), lower density (10⁶-10⁷/mL)

✅ **Practical considerations:**
- Batch dominant for GMP production (quality control, documentation)
- Fed-batch for high-value products (antibodies, biologics)
- Continuous for research or high-volume metabolites

---

## 🔗 Connection to Chapter 5

This exercise connects to:
- **Section 5.4.1:** Principles of scalable cell expansion
- **Section 5.4.2:** From cell expansion to cell-based products
- **Figure 5.1:** Continuum of bioprocesses
- **Scale-up strategies:** Batch size vs. production frequency

Related exercises:
- **Exercise 1:** Bioreactor selection (which reactor for which mode?)
- **Exercise 2:** Oxygen transfer (OUR increases with cell density)
- **Exercise 6:** Economics (compare costs across modes)

---

*Developed for Master's-level Bioengineering students*  
*Vrije Universiteit Brussel - Biofabrication Course 2025*